# Simple DeGroot Social Learning Model

In [334]:
import numpy as np
import typing
from sys import float_info  # default value for testing convergence
from numpy.typing import ArrayLike
from fractions import Fraction as fract  # format decimals as fractions

Take a society of 
$n$ agents where everybody has an opinion on a subject, represented by a vector of probabilities
$$
p(0) = (p_1(0), \dots, p_n(0))
$$

Agents obtain no new information based on which they can update their opinions but they communicate with other agents. Links between agents (who knows whom) and the weight they put on each other's opinions is represented by a *trust matrix*
 $T$ where 
$ T_{ij} $ is the weight that agent 
$i$
puts on agent 
$j$'s opinion. The trust matrix is thus in a one-to-one relationship with a weighted, directed graph where there is an edge between 
$i$ and 
$j$ iff $T_{ij} > 0$. 

The trust matrix is stochastic, its rows consists of nonnegative real numbers, with each row summing to 1.


Source: https://en.wikipedia.org/wiki/DeGroot_learning

We implement the mode, as a `DeGroot` simple class which takes as inputs:
    - a *belief vector*, a `numpy` array of shape $N$ representing the initial state of the population's beliefs 
    - a *trust matrix*, i.e. the transition matrix as a `numpy` array of floats of shape $NxN$.
    



In [341]:
class DeGroot:
    """
    A class implementing DeGroot social learning model

    Args:
        beliefs(numpy array): a vector representing the initial beliefs
        of the population
        trust_matrix(numpy array): the model's transitions matrix

    Attributes:
        beliefs(numpy array): The current state of the agents beliefs
    """

    def __init__(self, beliefs: ArrayLike, trust_matrix: ArrayLike) -> None:
        """
        Initializes object after checking that inputs are correct.
        """

        if len(beliefs) != len(trust_matrix):
            error_msg = f"incompatible dimensions for beliefs vector and trust matrix.\n Belief vector has shape: {beliefs.shape} and the trust matrix has shape {trust_matrix.shape}"
            raise ValueError(error_msg)
        if trust_matrix.shape[0] != trust_matrix.shape[1]:
            error_msg = f"trust matrix must have shape: {len(beliefs)} x {len(beliefs)}.\n Received shape {trust_matrix.shape}"
            raise ValueError(error_msg)
        if np.any([0 >= belief >= 1 for belief in beliefs]):
            error.msg = "sum of beliefs vector is not equal to 1"
            raise ValueError(error_msg)
        if np.any([sum(row) != 1 for row in trust_matrix]):
            raise ValueError("all rows of the trust matrix should have a sum of 1")

        self.beliefs = beliefs
        self._trust = trust_matrix

    def __str__(self):
        fracts = list(
            map(lambda n: fract(n).limit_denominator(100), self.beliefs.T.tolist())
        )
        fracts = "\n".join([f"{n.denominator}/{n.numerator}" for n in fracts])
        return fracts

    def _time_step(self) -> ArrayLike:
        # updates the model
        self.beliefs = self.beliefs @ self._trust
        return self.beliefs

    def iterate(
        self, no_iters: int = 1000, tolerance_level: float = float_info.epsilon
    ) -> (bool, list[ArrayLike]):
        if no_iters < 1:
            raise ValueError(
                f"number of iterations must be 1 or greater. Got {no_iters}"
            )
        history = []
        
        for i in range(no_iters):
            old_vector = self.beliefs
            history.append(old_vector)
            new_vector = self._time_step()
            if np.all(abs(new_vector - old_vector) < tolerance_level):
                print(f"Convergence reached at the {i}th time_step\n")
                return (True, history)
        print("No Convergence\n")
        return (False, history)

In [342]:
belief_vector = np.array([1, 0, 0])
trust_matrix = np.array([[0, 0.5, 0.5], [1, 0, 0], [0, 1, 0]])

jackson_example = DeGroot(belief_vector, trust_matrix)

In [340]:
jackson_example.iterate()
print(jackson_example)

Convergence reached at the 105th time_step

5/2
5/2
5/1
